In [1]:
from utils import load_my_data
from utils import extract_DenseSift_descriptors
from utils import build_codebook
from utils import input_vector_encoder
from classifier import svm_classifier
from keras.datasets import mnist
import spm

import numpy as np
import argparse

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
print X_train.shape

(60000, 28, 28)


In [4]:


X, y = X_train, y_train

X_small = []
y_small = []
for digit in range(0,10):
    x_tmp = [X[i] for i in range(60000) if y[i] == digit]
    y_tmp = [y[i] for i in range(60000) if y[i] == digit]
    X_small = X_small + x_tmp[:100]
    y_small = y_small + y_tmp[:100]

X = X_small
y = y_small

print "Codebook Size: {:d}".format(spm.VOC_SIZE)
print "Pyramid level: {:d}".format(spm.PYRAMID_LEVEL)
print "Dense SIFT feature extraction"

Codebook Size: 200
Pyramid level: 2
Dense SIFT feature extraction


In [5]:
x_feature = [extract_DenseSift_descriptors(img) for img in X]
x_kp, x_des = zip(*x_feature)
xt_feature = [extract_DenseSift_descriptors(img) for img in X_test]
xt_kp, xt_des = zip(*xt_feature)

In [ ]:
%%time
## SIFT based matching only for experimentation
trx = np.array([np.array(a).flatten() for a in x_des])
tex = np.array([np.array(a).flatten() for a in xt_des])
svm_classifier(trx, y, tex, y_test)

In [ ]:
%%time
print "Building the codebook, it will take some time"
codebook = build_codebook(x_des, spm.VOC_SIZE)
#import cPickle
#with open('./data/codebook_spm.pkl','w') as f:
#    cPickle.dump(codebook, f)

In [ ]:
%%time

print "Spatial Pyramid Matching encoding"
X = [spm.spatial_pyramid_matching(X[i],
                              x_des[i],
                              codebook,
                              level=spm.PYRAMID_LEVEL)
                              for i in xrange(len(x_des))]

In [ ]:
%%time
X_test = [spm.spatial_pyramid_matching(X_test[i],
                              xt_des[i],
                              codebook,
                              level=spm.PYRAMID_LEVEL)
                              for i in xrange(len(xt_des))]

In [ ]:
%%time
X = np.asarray(X)
X_test = np.asarray(X_test)
svm_classifier(X, y, X_test, y_test)